In [8]:
pip install kaggle


In [9]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection-Kaggle API

In [10]:
kaggle_dictionary = json.load(open("/content/kaggle.json"))

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [13]:
# setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [14]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other


In [15]:
!ls

drive  imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [16]:
# unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [17]:
!ls

 drive	'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


Loading the Dataset

In [18]:
data= pd.read_csv("/content/IMDB Dataset.csv")

In [19]:
data.shape

(50000, 2)

In [20]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [21]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [22]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [23]:
data.replace({"sentiment":{"positive":1, "negative":0}},inplace=True)

<ipython-input-23-9c1542598521>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1, "negative":0}},inplace=True)


In [24]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [25]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [26]:
# split data into training data and test data
train_data,test_data =train_test_split(data,test_size=0.2,random_state=42)

In [27]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Preprocessing

In [28]:
#Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [29]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [30]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [31]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [32]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


**LSTM - Long Short term Memory**

In [33]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))
import numpy as np
dummy_input = np.zeros((1, 100))  # (batch_size=1, sequence_length=100)
model.build(dummy_input.shape)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [34]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (1, 100, 128)               │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (1, 128)                    │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (1, 1)                      │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
5000*128

640000

In [36]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

**Training the model**

In [37]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 389ms/step - accuracy: 0.7234 - loss: 0.5348 - val_accuracy: 0.8519 - val_loss: 0.3482
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 398ms/step - accuracy: 0.8481 - loss: 0.3675 - val_accuracy: 0.8514 - val_loss: 0.3468
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 200s 395ms/step - accuracy: 0.8793 - loss: 0.3038 - val_accuracy: 0.8537 - val_loss: 0.3469
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 196s 382ms/step - accuracy: 0.8952 - loss: 0.2638 - val_accuracy: 0.8650 - val_loss: 0.3367
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 208s 395ms/step - accuracy: 0.8880 - loss: 0.2721 - val_accuracy: 0.8686 - val_loss: 0.3158


**Model Evaluation**

In [38]:
model.save('my_model.h5')

In [39]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [40]:
loss,accuracy =model.evaluate(X_test,Y_test)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 115ms/step - accuracy: 0.8719 - loss: 0.3031
Loss: 0.30245015025138855
Accuracy: 0.8770999908447266


**Building a predictive system**

In [41]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [42]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step
The sentiment of the review is: positive


In [43]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
The sentiment of the review is: negative


In [44]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
The sentiment of the review is: negative


In [45]:
predict_sentiment("A trilling adventure with stunning visual")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step


'positive'

In [46]:
new_review = "This movie was well."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
The sentiment of the review is: negative


In [47]:
predict_sentiment("movie was okay")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step


'negative'

In [48]:
predict_sentiment("movie was long and slow")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step


'negative'

In [49]:
predict_sentiment("movie was very boring and not entertaining")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step


'negative'